L' analisi dell'energia ci può essere utile per permetterci di eliminare i file audio che sono meno significativi. Il seguente codice crea una classifica di audio che vanno dal meno significativo al più significativo. Lo si fa solo per le cartelle che superano la soglia di 8 mila file.


In [4]:
import os
import librosa
import numpy as np
import pandas as pd

# Definizione del percorso del dataset
dataset_dir = 'NuovoDataset/Segmented'
target_dir = os.path.join(dataset_dir, 'Target')
non_target_dir = os.path.join(dataset_dir, 'Non-Target')

# Creazione della directory per salvare le classifiche
ranking_dir = os.path.join(dataset_dir, 'Rankings')
os.makedirs(ranking_dir, exist_ok=True)

# Funzione per calcolare l'energia del segnale audio
def calculate_energy(file_path):
    y, sr = librosa.load(file_path)
    energy = np.sum(y**2) / len(y)
    return energy

# Funzione per generare la classifica dei file audio per una determinata directory
def rank_audio_files(directory, subdir_name):
    audio_files = []
    print(f"Ranking files in directory: {directory}")  # Debug
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                try:
                    energy = calculate_energy(file_path)
                    audio_files.append((file_path, energy))
                except Exception as e:
                    print(f"Errore nel calcolo dell'energia per il file {file_path}: {e}")

    if audio_files:
        # Creare un DataFrame con i file audio e le loro energie
        df = pd.DataFrame(audio_files, columns=['file_path', 'energy'])
        
        # Ordinare i file audio per energia crescente
        df_sorted = df.sort_values(by='energy')
        
        # Salvare la classifica in un file CSV nella directory delle classifiche
        ranking_subdir = os.path.join(ranking_dir, subdir_name)
        os.makedirs(ranking_subdir, exist_ok=True)
        csv_path = os.path.join(ranking_subdir, 'audio_file_ranking.csv')
        df_sorted.to_csv(csv_path, index=False)
        print(f"Saved ranking to {csv_path}")  # Debug
    else:
        print(f"No audio files found in {directory}")  # Debug

# Funzione per elaborare solo le sottocartelle con più di 8.000 file audio
def process_large_subdirectories(main_directory):
    for category in ['Target', 'Non-Target']:
        category_dir = os.path.join(main_directory, category)
        for subdir in os.listdir(category_dir):
            subdir_path = os.path.join(category_dir, subdir)
            if os.path.isdir(subdir_path):
                # Conta il numero di file audio nella sottocartella
                num_files = sum([len(files) for r, d, files in os.walk(subdir_path) if any(file.endswith('.wav') for file in files)])
                print(f"Sottocartella: {subdir_path}, Numero di file: {num_files}")  # Debug
                if num_files > 8000:
                    print(f"Elaborazione della sottocartella: {subdir_path}, Numero di file: {num_files}")
                    rank_audio_files(subdir_path, subdir)

# Esegui la funzione
process_large_subdirectories(dataset_dir)

print("Classifica dei file audio per sottocartelle con più di 8.000 file completata.")


Sottocartella: NuovoDataset/Segmented\Target\Acoustic Harassment Devices, Numero di file: 2
Sottocartella: NuovoDataset/Segmented\Target\Acoustic release, Numero di file: 3
Sottocartella: NuovoDataset/Segmented\Target\Cargo, Numero di file: 9671
Elaborazione della sottocartella: NuovoDataset/Segmented\Target\Cargo, Numero di file: 9671
Ranking files in directory: NuovoDataset/Segmented\Target\Cargo
Saved ranking to NuovoDataset/Segmented\Rankings\Cargo\audio_file_ranking.csv
Sottocartella: NuovoDataset/Segmented\Target\Echosounder, Numero di file: 14
Sottocartella: NuovoDataset/Segmented\Target\Explosion, Numero di file: 2
Sottocartella: NuovoDataset/Segmented\Target\Fishing pinger, Numero di file: 2
Sottocartella: NuovoDataset/Segmented\Target\Passengership, Numero di file: 11728
Elaborazione della sottocartella: NuovoDataset/Segmented\Target\Passengership, Numero di file: 11728
Ranking files in directory: NuovoDataset/Segmented\Target\Passengership
Saved ranking to NuovoDataset/Segme